In [1]:
import os
import pickle
import numpy as np
from PIL import Image
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt

In [2]:
model_path = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\02958343\\02958343\\1a0bc9ab92c915167ae33d942430658c\\model.obj"

In [3]:
vertex_list = []
face_list = []

In [4]:
def get_vertex_face(model_path):
    vertex_list = []
    face_list = []
    with open(model_path, "r") as f:
        for line in f:
            if line.startswith("#") or line.startswith("mtllib") or line.startswith("g") or line.startswith("usemtl"):
                continue
            elif line.startswith("v "):
                #print(line.split())
                x = float(line.split(" ")[1])
                y = float(line.split(" ")[2])
                z = float(line.split(" ")[3])
                vertex_list.append([x, y, z])
                continue
            elif line.startswith("f "):
                v1 = int(line.split(" ")[1].split("/")[0])
                v2 = int(line.split(" ")[2].split("/")[0])
                v3 = int(line.split(" ")[3].split("/")[0])
                face_list.append([v1, v2, v3])
                continue
    return vertex_list, face_list

In [21]:
### done, successfully reconstruct a 3d model without texture
with open("./test_ply.ply", "w") as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex " + str(len(vertex_list)) + "\n")
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    f.write("element face " + str(len(face_list)) + "\n")
    f.write("property list uint8 uint32 vertex_indices\n")
    f.write("end_header\n")
    for vertex in vertex_list:
        f.write(str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + "\n")
    for face in face_list:
        f.write(str(3) + " " + str(face[0] - 1) + " " + str(face[1] - 1) + " " + str(face[2] - 1) + "\n")

In [44]:
model_list = []
for file in os.listdir("./rotate"):
    if file.startswith("model_name_"):
        print(os.path.join("./rotate", file))
        with open(os.path.join("./rotate", file), "rb") as f:
            model_list_partial = pickle.load(f)
        for model in model_list_partial:
            model_list.append(model.split("/")[-2])
    #break

precision_constant = 10000
    
model_path = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\02958343\\02958343"
for modelidx, model in enumerate(model_list):
    if modelidx not in [2]:
        continue
    #os.system("blender --background --python 3dloc_2dcoor.py -- --views 18 " + os.path.join(model_path, model, "model.obj"))
    with open("./vertex2pixel", "rb") as f:
        vertex2pixel = pickle.load(f)
    with open("./pixel2vertex", "rb") as f:
        pixel2vertex = pickle.load(f)
    vertex2color = OrderedDict()
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            realimg = np.array(Image.open(real_img_path).convert("RGB"))
            realimg_mask = np.array(Image.open(real_img_path))[:, :, 3].reshape([128, 128])
            syn_img_path = "./rotate/rotate/" + str(modelidx * 1000 + az * 10 + el) + ".0.jpg"
            synimg = np.array(Image.open(syn_img_path))
            '''
            for vertex_pixel_pair in vertex2pixel["az_" + str(az * 20) + "_el_" + str(el * 10)]:
                if not repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int)) in vertex2color:
                    vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))] = []
                vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))].append(realimg[vertex_pixel_pair[3], vertex_pixel_pair[3]])
            '''
            for i in range(128):
                for j in range(128):
                    if pixel2vertex[az, el, i, j, 0] != 1234567.0:
                        if realimg_mask[i, j] < 255 / 2:
                            print("pass!")
                            continue
                        if (synimg[i, j] == 0).any():
                            continue
                        if not repr((10000 * pixel2vertex[az, el, i, j, 1:]).astype(int)) in vertex2color:
                            vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))] = []
                        #vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(realimg[i, j])
                        vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append([synimg[i, j], pixel2vertex[az, el, i, j, 0]])
            
    vertex_list, face_list = get_vertex_face(os.path.join(model_path, model, "model.obj"))
    color_list = np.zeros(shape=(len(vertex_list), 3)).astype(int)
    for vertexidx, vertex in enumerate(vertex_list):
        if repr((np.array(vertex) * precision_constant).astype(int)) in vertex2color:
            candidate_color_list = []
            candidate_dist_list = []
            for idxcolor in range(len(vertex2color[repr((np.array(vertex) * precision_constant).astype(int))])):
                candidate_color_list.append(vertex2color[repr((np.array(vertex) * precision_constant).astype(int))][idxcolor][0])
                candidate_dist_list.append(vertex2color[repr((np.array(vertex) * precision_constant).astype(int))][idxcolor][1])
            candidate_color_list = np.array(candidate_color_list, dtype=np.int16)
            candidate_dist_list = np.array(candidate_dist_list, dtype=np.int16)
            distsort = np.argsort(-np.array(candidate_dist_list))
            if len(distsort) > 5:
                distsort = distsort[:5]
            candidate_color = np.zeros(shape=3)
            candidate_weight = 0
            for idxdist in range(len(distsort)):
                candidate_color += (idxdist + 1) * candidate_color_list[distsort[idxdist]]
                candidate_weight += (idxdist + 1)
            color_list[vertexidx] = candidate_color / candidate_weight
            '''
            candidate_color_list = vertex2color[repr((np.array(vertex) * precision_constant).astype(int))]
            mean = np.mean(candidate_color_list, axis=0).astype(int)
            std = np.std(candidate_color_list, axis=0).astype(int)
            for coloridx, candidate_color in enumerate(candidate_color_list):
                if (candidate_color > mean + 1 * std).all() or (candidate_color < mean - 1 * std).all():
                    candidate_color_list[coloridx][:3] = mean
            color = np.mean(candidate_color_list, axis=0).astype(int)
            color_list[vertexidx] = color
            '''

    vertex2neighbor = OrderedDict()
    for face in face_list:
        v1, v2, v3 = face[0] - 1, face[1] - 1, face[2] - 1;
        if str(v1) not in vertex2neighbor:
            vertex2neighbor[str(v1)] = set()
        if str(v2) not in vertex2neighbor:
            vertex2neighbor[str(v2)] = set()
        if str(v3) not in vertex2neighbor:
            vertex2neighbor[str(v3)] = set()
        vertex2neighbor[str(v1)].add(v2)
        vertex2neighbor[str(v1)].add(v3)
        vertex2neighbor[str(v2)].update([v1, v3])
        vertex2neighbor[str(v3)].update([v1, v2])
    cnt_loop = 0
    while True:
        cnt_loop += 1
        all_vertex_not_colored = True
        for vertexidx, vertex in enumerate(vertex_list):
            if str(vertexidx) not in vertex2neighbor:
                continue
            if np.sum(color_list[vertexidx]) < 10:
                color = np.array([0, 0, 0])
                cnt = 0
                for vertexx in vertex2neighbor[str(vertexidx)]:
                    if np.sum(color_list[vertexx]) >= 10:
                        color += color_list[vertexx]
                        cnt += 1
                if cnt != 0:
                    color_list[vertexidx] = np.array(color) / cnt
                all_vertex_not_colored = False
        if all_vertex_not_colored:
            break
        if cnt_loop >= 100:
            break
    
    cnt_loop = 2
    for loopidx in range(cnt_loop):
        for vertexidx, vertex in enumerate(vertex_list):
            if str(vertexidx) not in vertex2neighbor:
                continue
            if np.sum(color_list[vertexidx]) > 10:
                candidate_color_list = []
                for vertexx in vertex2neighbor[str(vertexidx)]:
                    candidate_color_list.append(color_list[vertexx])
                std = np.std(candidate_color_list, axis=0).astype(int)
                for coloridx, candidate_color in enumerate(candidate_color_list):
                    if (candidate_color > color_list[vertexidx] + 1 * std).all() or (candidate_color < color_list[vertexidx] - 1 * std).all():
                        candidate_color_list[coloridx][:3] = color_list[vertexidx]
                color = np.mean(candidate_color_list, axis=0).astype(int)
                color_list[vertexidx] = color

    with open("./test_ply_syn_mindist_propagation_distavg_mask_" + str(modelidx) + ".ply", "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write("element vertex " + str(len(vertex_list)) + "\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property uchar red\n")
        f.write("property uchar green\n")
        f.write("property uchar blue\n")
        f.write("element face " + str(len(face_list)) + "\n")
        f.write("property list uint8 uint32 vertex_indices\n")
        f.write("end_header\n")
        for vertexidx, vertex in enumerate(vertex_list):
            color = color_list[vertexidx]
            f.write(str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + " " + str(color[0]) + " " + str(color[1]) + " " + str(color[2]) + "\n")
        for face in face_list:
            f.write(str(3) + " " + str(face[0] - 1) + " " + str(face[1] - 1) + " " + str(face[2] - 1) + "\n")

./rotate\model_name_0
./rotate\model_name_1000
./rotate\model_name_2000
./rotate\model_name_3000
./rotate\model_name_5000
./rotate\model_name_6000
./rotate\model_name_7000
./rotate\model_name_8000
./rotate\model_name_9000
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

In [42]:
cnt = 0
for color in color_list:
    if np.sum(color) == 0.0:
        cnt += 1
print(cnt)

1397


In [35]:
if True:
    color_list = np.zeros(shape=(len(vertex_list), 3)).astype(int)
    for vertexidx, vertex in enumerate(vertex_list):
        if repr((np.array(vertex) * precision_constant).astype(int)) in vertex2color:
            candidate_color_list = []
            candidate_dist_list = []
            for idxcolor in range(len(vertex2color[repr((np.array(vertex) * precision_constant).astype(int))])):
                candidate_color_list.append(vertex2color[repr((np.array(vertex) * precision_constant).astype(int))][idxcolor][0])
                candidate_dist_list.append(vertex2color[repr((np.array(vertex) * precision_constant).astype(int))][idxcolor][1])
            candidate_color_list = np.array(candidate_color_list, dtype=np.int16)
            candidate_dist_list = np.array(candidate_dist_list, dtype=np.int16)
            mean = np.mean(candidate_color_list, axis=0).astype(int)
            std = np.std(candidate_color_list, axis=0).astype(int)
            for coloridx, candidate_color in enumerate(candidate_color_list):
                if (candidate_color > mean + 1 * std).all() or (candidate_color < mean - 1 * std).all():
                    candidate_color_list[coloridx] = mean
            distsort = np.argsort(-candidate_dist_list)
            candidate_color = np.zeros(shape=3)
            candidate_weight = 0
            for idxdist in range(len(distsort)):
                candidate_color += (idxdist + 1) * candidate_color_list[distsort[idxdist]]
                candidate_weight += (idxdist + 1)
                print(candidate_color, (idxdist + 1), candidate_color_list[distsort[idxdist]])
            color_list[vertexidx] = candidate_color / candidate_weight
            print(candidate_color_list)
            print(distsort)
            print(color_list[vertexidx], vertex2color[repr((np.array(vertex) * precision_constant).astype(int))])
            break

[156. 126. 134.] 1 [156 126 134]
[436. 282. 320.] 2 [140  78  93]
[[156 126 134]
 [140  78  93]]
[0 1]
[145  94 106] [[array([156, 126, 134], dtype=uint8), 1.2737650871276855], [array([140,  78,  93], dtype=uint8), 1.2085500955581665]]


In [37]:
vertex2color

OrderedDict([('array([-290, 1292, 1138])',
              [[array([36, 25, 29], dtype=uint8), 1.0861001014709473],
               [array([32, 32, 32], dtype=uint8), 1.0830427408218384],
               [array([141,  69,  73], dtype=uint8), 1.0786718130111694],
               [array([147,  88,  94], dtype=uint8), 1.1103620529174805],
               [array([139,  76,  67], dtype=uint8), 1.1640782356262207],
               [array([154,  61,  71], dtype=uint8), 1.064557433128357]]),
             ('array([-477, 1318, 1034])',
              [[array([42, 31, 35], dtype=uint8), 1.096513032913208],
               [array([145,  65,  66], dtype=uint8), 1.0751935243606567],
               [array([169,  58,  64], dtype=uint8), 1.057666301727295],
               [array([113,  54,  56], dtype=uint8), 1.0688154697418213],
               [array([162, 153, 148], dtype=uint8), 1.2809172868728638],
               [array([105,  38,  29], dtype=uint8), 1.2567963600158691],
               [array([34, 14, 16], 

In [ ]:
# test_ply_real: threshold = 1.0, with min dist, real pixel, const=10000
# test_plt_syn: threshold = 1.0, with min dist, syn pixel, const=10000
# with min dist:threshold = 1.0 or 0.2 -> blurry, but why? 
#    - instable ray_cast?
# threshold = 0.2: more vertices left blank, with const=1e6
#    - fixed with const=10000, see test_ply_real_threshold20
# 0615: epoch=3, lossGB about 2.0, run with real depth image

In [104]:
### test mask
for modelidx, model in enumerate(model_list):
    #os.system("blender --background --python 3dloc_2dcoor.py -- --views 18 " + os.path.join(model_path, model, "model.obj"))
    with open("./vertex2pixel", "rb") as f:
        vertex2pixel = pickle.load(f)
    with open("./pixel2vertex", "rb") as f:
        pixel2vertex = pickle.load(f)
    vertex2color = OrderedDict()
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            realimg = np.array(Image.open(real_img_path).convert("RGB"))
            realimg_mask = np.array(Image.open(real_img_path))[:, :, 3].reshape([128, 128])
            syn_img_path = "./rotate/rotate/" + str(modelidx * 1000 + az * 10 + el) + ".0.jpg"
            synimg = np.array(Image.open(syn_img_path))
            '''
            for vertex_pixel_pair in vertex2pixel["az_" + str(az * 20) + "_el_" + str(el * 10)]:
                if not repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int)) in vertex2color:
                    vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))] = []
                vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))].append(realimg[vertex_pixel_pair[3], vertex_pixel_pair[3]])
            '''
            for i in range(128):
                for j in range(128):
                    if pixel2vertex[az, el, i, j, 0] != 1234567.0:
                        if realimg_mask[i, j] < 255 / 2:
                            print("pass!")
                            continue
                        if (synimg[i, j] == 0).any():
                            continue
                        if not repr((10000 * pixel2vertex[az, el, i, j, 1:]).astype(int)) in vertex2color:
                            vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))] = []
                        #vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(realimg[i, j])
                        vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(synimg[i, j])
    vertex_list, face_list = get_vertex_face(os.path.join(model_path, model, "model.obj"))
    color_list = np.zeros(shape=(len(vertex_list), 3)).astype(int)
    for vertexidx, vertex in enumerate(vertex_list):
        if repr((np.array(vertex) * precision_constant).astype(int)) in vertex2color:
            candidate_color_list = vertex2color[repr((np.array(vertex) * precision_constant).astype(int))]
            mean = np.mean(candidate_color_list, axis=0).astype(int)
            std = np.std(candidate_color_list, axis=0).astype(int)
            for coloridx, candidate_color in enumerate(candidate_color_list):
                if (candidate_color > mean + 1 * std).all() or (candidate_color < mean - 1 * std).all():
                    candidate_color_list[coloridx] = mean
            color = np.mean(candidate_color_list, axis=0).astype(int)
            color_list[vertexidx] = color
    if modelidx >= 0:
        break

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!


In [107]:
cnt = 0
for color in color_list:
    if np.sum(color) <= 10.0:
        cnt += 1
print(cnt)

3157


In [16]:
candidate_dist_list

[[array([140,  78,  93], dtype=uint8), 1.2085500955581665],
 [array([140,  78,  93], dtype=uint8), 1.2085500955581665]]

In [6]:
np.argsort(syn_loss)[:10]

array([ 319, 1442,  129,  333, 1574, 1941, 1428,  978,  779, 1670],
      dtype=int64)

In [12]:
model_list[2]

'60b73f29f7a245f8297fefe981bdb889'